# Generate Knowlege Graph With Trained Model

## Retrive Triplets
Load trained model

In [ ]:
import os
import json

import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm

In [ ]:
torch.__version__

In [ ]:
def load_model(model_dir, epoch, device):
    s_m = torch.load(os.path.join(model_dir, "s_{}.pkl".format(epoch)), map_location=device)
    po_m = torch.load(os.path.join(model_dir, "po_{}.pkl".format(epoch)), map_location=device)
    # reload the model with DataParallel (this will 
    # be helpful when num of GPUs changes)
    s_m = nn.DataParallel(s_m.module)
    po_m = nn.DataParallel(po_m.module)
    return s_m, po_m

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_epoch = 210
model_dir = 'models_real'
s_m, po_m = load_model(model_dir, model_epoch, device)

Extract triplets from dev data (currently )

In [ ]:
# This is a bad sequential implementation
def extract_items(text_in, s_m, po_m):
    R = []
    _s = [char2id.get(c, 1) for c in text_in]
    _s = np.array([_s])
    _k1, _k2, t, t_max, mask = s_m(torch.LongTensor(_s).to(device))
    _k1, _k2 = _k1[0, :, 0], _k2[0, :, 0]
    _kk1s = []
    for i, _kk1 in enumerate(_k1):
        if _kk1 > 0.5:
            _subject = ''
            for j, _kk2 in enumerate(_k2[i:]):
                if _kk2 > 0.5:
                    _subject = text_in[i: i+j+1]
                    break
            if _subject:
                _k1, _k2 = torch.LongTensor([[i]]), torch.LongTensor(
                    [[i+j]])  # np.array([i]), np.array([i+j])
                _o1, _o2 = po_m(t.to(device), t_max.to(
                    device), _k1.to(device), _k2.to(device))
                _o1, _o2 = _o1.cpu().data.numpy(), _o2.cpu().data.numpy()

                _o1, _o2 = np.argmax(_o1[0], 1), np.argmax(_o2[0], 1)

                for i, _oo1 in enumerate(_o1):
                    if _oo1 > 0:
                        for j, _oo2 in enumerate(_o2[i:]):
                            if _oo2 == _oo1:
                                _object = text_in[i: i+j+1]
                                _predicate = id2predicate[_oo1]
                                # print((_subject, _predicate, _object))
                                R.append((_subject, _predicate, _object))
                                break
        _kk1s.append(_kk1.data.cpu().numpy())
    _kk1s = np.array(_kk1s)
    return list(set(R))

Load dev data, and corresponding schemas

In [ ]:
dev_path = 'generated/dev_data_me.json'
dev_data = json.load(open(dev_path))
generated_char_path = 'generated/all_chars_me.json'
id2char, char2id = json.load(open(generated_char_path))
generated_schema_path =  'generated/schemas_me.json'
id2predicate, predicate2id = json.load(open(generated_schema_path))
id2predicate = {int(i): j for i, j in id2predicate.items()}

Write to `pandas` frame first, then write to a csv file.

In [ ]:
import pandas as pd
import csv

In [ ]:
df = pd.DataFrame({'subject':[], 'predicate':[], 'object':[]})
for d in tqdm(iter(dev_data)):
    items = extract_items(d['text'], s_m, po_m)
    for item in items:
        df.loc[len(df)] = item

print("num of extracted relations from dev set is:", len(df))

In [ ]:
df.to_csv('generated/triplets.csv', index=False, header=False)

Create knowledge graph with saved triplets

In [ ]:
# check the length of existing predicates
len(set(df['predicate']))